In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [6]:
df = pd.read_csv('enwiki-20190320-words-frequency.txt', sep = ' ', names = ['word', 'frequency'])

In [7]:
df.head()

,word,frequency
0,the,151983633
1,of,71874676
2,and,62210193
3,in,62004799
4,to,43364193


In [21]:
df1 = df.sort_values(by = ['frequency'], ascending = False)[['word']]

In [22]:
df1.head()

,word
0,the
1,of
2,and
3,in
4,to


In [23]:
df1.to_csv('words-by-frequency.txt', index = False, header = False)

In [25]:
from math import log

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = open("words-by-frequency.txt", encoding = 'utf-8').read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [29]:
s = 'thisalgorithmissuperefficientatfindingspace'
print(infer_spaces(s))

this algorithm is superefficient at finding space


In [32]:
for i in tqdm(range(50000)):
    infer_spaces(s)